In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras as kr
from keras.preprocessing.image import ImageDataGenerator

In [2]:
BASE_PATH = './drive/MyDrive/Jupyter/datasets/birds'
IMAGE_SIZE = (224, 224)
INPUT_SHAPE = (*IMAGE_SIZE, 3)
NUM_CLASSES = 500
BATCH_SIZE = 64